# Stegasus

## Commons

In [ ]:
%pip install icecream
from icecream import ic

In [ ]:
ROOT_DIR = '.'

In [ ]:
#@title random_bit_stream
import random

def random_bit_stream(length=None):
    """Return a random string of zeros and ones of the given length (default: random integer between 0 and 100)."""
    if length is None:
        length = random.randint(0, 100)
    return ''.join(str(random.randint(0, 1)) for _ in range(length))
def int_to_binary_string(n: int, length: int):
    binary_str = bin(n)[2:]  # convert to binary string, remove '0b' prefix
    padded_str = binary_str.rjust(length, '0')  # pad with zeros to length
    return padded_str

In [ ]:
from dataclasses import dataclass, field

In [ ]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

## Frustratingly Simple BERT

In [ ]:
#@title new masking approach
import spacy

def extract_pos(text):
    """Extract the start and end positions of verbs, nouns, and adjectives in the given text."""
    # Load the spaCy English model
    nlp = spacy.load('en_core_web_sm')
    
    # Parse the text with spaCy
    doc = nlp(text)
    
    # Create a dictionary to store the start and end positions of each POS tag
    pos_dict = {
        'VERB': [],
        'NOUN': [],
        'ADJ': []
    }
    
    # Loop through each token in the parsed text
    for token in doc:
        if token.pos_ in pos_dict:
            # If the token's POS tag is a verb, noun, or adjective, add its start and end positions to the dictionary
            pos_dict[token.pos_].append((token.idx, token.idx + len(token)))
    
    return pos_dict


In [ ]:
# @title Setup Installs Imports
%pip install transformers
%pip install icecream
import nltk

nltk.download('stopwords')
from typing import List, Tuple, Union
from io import StringIO
from icecream import ic 

from nltk.corpus import stopwords
import torch
from torch import Tensor
import torch.nn.functional as F
from transformers import BertTokenizer, BertForMaskedLM
from transformers.tokenization_utils import PreTrainedTokenizer


In [ ]:
#@title Class Definition

@dataclass
class MaskedStegoResult:
  encoded_text: str
  encoded_bytes: str
  remaining_bytes: str

class MaskedStego:
    debugging = False
    def __init__(self, model_name_or_path: str = 'bert-base-cased') -> None:
        self._tokenizer: PreTrainedTokenizer = BertTokenizer.from_pretrained(model_name_or_path)
        self._model = BertForMaskedLM.from_pretrained(model_name_or_path)
        self._STOPWORDS: List[str] = stopwords.words('english')

    def __call__(self, cover_text: str, message: str, mask_interval: int = 3, score_threshold: float = 0.01) -> MaskedStegoResult:
        assert set(message) <= set('01')
        message_io = StringIO(message)
        if self.debugging:
          ic(message_io)
        processed = self._preprocess_text(cover_text, mask_interval)
        if self.debugging:
          ic(processed)
        input_ids = processed['input_ids']
        # ic(input_ids)
        masked_ids = processed['masked_ids']
        # ic(masked_ids)
        sorted_score, indices = processed['sorted_output']
        # ic(sorted_score, indices)
        for i_token, token in enumerate(masked_ids):
            if self.debugging:
              ic(i_token, token)
            if self.debugging:
              ic(self._tokenizer.mask_token_id)
            if token != self._tokenizer.mask_token_id:
                continue
            ids = indices[i_token]
            if self.debugging:
              ic(ids)
            scores = sorted_score[i_token]
            if self.debugging:
              ic(scores)
            candidates = self._pick_candidates_threshold(ids, scores, score_threshold)
            # ic(candidates)
            if self.debugging:
              ic(self._tokenizer.convert_ids_to_tokens(candidates))
            if self.debugging:
              ic(len(candidates) < 2)
            if len(candidates) < 2:
                continue
            replace_token_id = self._block_encode_single(candidates, message_io).item()
            if self.debugging:
              ic(replace_token_id)
            if self.debugging:
              ic('replace', replace_token_id, self._tokenizer.convert_ids_to_tokens([replace_token_id]))
            input_ids[i_token] = replace_token_id
        encoded_message: str = message_io.getvalue()[:message_io.tell()]
        if self.debugging:
          ic(encoded_message)
        message_io.close()
        stego_text = self._tokenizer.decode(input_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        if self.debugging:
          ic(stego_text)
        return MaskedStegoResult(encoded_text=stego_text,encoded_bytes=encoded_message,remaining_bytes=message[len(encoded_message):])

    def decode(self, stego_text: str, mask_interval: int = 3, score_threshold: float = 0.005) -> str:
        decoded_message: List[str] = []
        processed = self._preprocess_text(stego_text, mask_interval)
        input_ids = processed['input_ids']
        masked_ids = processed['masked_ids']
        sorted_score, indices = processed['sorted_output']
        for i_token, token in enumerate(masked_ids):
            if token != self._tokenizer.mask_token_id:
                continue
            ids = indices[i_token]
            scores = sorted_score[i_token]
            candidates = self._pick_candidates_threshold(ids, scores, score_threshold)
            if len(candidates) < 2:
                continue
            chosen_id: int = input_ids[i_token].item()
            decoded_message.append(self._block_decode_single(candidates, chosen_id))

        return ''.join(decoded_message)

    def _preprocess_text(self, sentence: str, mask_interval: int) -> dict:
        encoded_ids = self._tokenizer([sentence], return_tensors='pt').input_ids[0]
        masked_ids = self._mask(encoded_ids.clone(), mask_interval)
        sorted_score, indices = self._predict(masked_ids)
        return { 'input_ids': encoded_ids, 'masked_ids': masked_ids, 'sorted_output': (sorted_score, indices) }

    def _mask(self, input_ids: Union[Tensor, List[List[int]]], mask_interval: int) -> Tensor:
        length = len(input_ids)
        tokens: List[str] = self._tokenizer.convert_ids_to_tokens(input_ids)
        offset = mask_interval // 2 + 1
        mask_count = offset
        for i, token in enumerate(tokens):
            # Skip initial subword
            if i + 1 < length and self._is_subword(tokens[i + 1]): continue
            if not self._substitutable_single(token): continue
            if mask_count % mask_interval == 0:
                input_ids[i] = self._tokenizer.mask_token_id
            mask_count += 1
        return input_ids

    def _predict(self, input_ids: Union[Tensor, List[List[int]]]):
        self._model.eval()
        with torch.no_grad():
            output = self._model(input_ids.unsqueeze(0))['logits'][0]
            softmaxed_score = F.softmax(output, dim=1)  # [word_len, vocab_len]
            return softmaxed_score.sort(dim=1, descending=True)

    def _encode_topk(self, ids: List[int], message: StringIO, bits_per_token: int) -> int:
        k = 2**bits_per_token
        candidates: List[int] = []
        for id in ids:
            token = self._tokenizer.convert_ids_to_tokens(id)
            if not self._substitutable_single(token):
                continue
            candidates.append(id)
            if len(candidates) >= k:
                break
        return self._block_encode_single(candidates, message)

    def _pick_candidates_threshold(self, ids: Tensor, scores: Tensor, threshold: float) -> List[int]:
        filtered_ids: List[int] = ids[scores >= threshold]
        def filter_fun(idx: Tensor) -> bool:
            return self._substitutable_single(self._tokenizer.convert_ids_to_tokens(idx.item()))
        return list(filter(filter_fun, filtered_ids))

    def _substitutable_single(self, token: str) -> bool:
        if self._is_subword(token): return False
        if token.lower() in self._STOPWORDS: return False
        if not token.isalpha(): return False
        return True

    @staticmethod
    def _block_encode_single(ids: List[int], message: StringIO) -> int:
        assert len(ids) > 0
        if len(ids) == 1:
            return ids[0]
        capacity = len(ids).bit_length() - 1
        bits_str = message.read(capacity)
        if len(bits_str) < capacity:
            padding: str = '0' * (capacity - len(bits_str))
            bits_str = bits_str + padding
            message.write(padding)
        index = int(bits_str, 2)
        return ids[index]

    @staticmethod
    def _block_decode_single(ids: List[int], chosen_id: int) -> str:
        if len(ids) < 2:
            return ''
        capacity = len(ids).bit_length() - 1
        index = ids.index(chosen_id)
        return format(index, '0' + str(capacity) +'b')

    @staticmethod
    def _is_subword(token: str) -> bool:
        return token.startswith('##')
masked_stego = MaskedStego()

In [ ]:
#@title Example

# print(masked_stego.decode(args.text, args.mask_interval, args.score_threshold))
print(masked_stego("The quick brown fox jumps over the lazy dog.",'010101010101', 3, 0.01))
print(masked_stego.decode("The quick red fox jumps over the poor dog.", 3, 0.01))
# print(masked_stego("The quick brown fox jumps over the lazy dog. and said boom you lazy dog stay back",'010101010101', 3, 0.01))


## Typoceros

In [ ]:
!rm -r Stegasus

In [ ]:
!git clone https://github.com/NasoohOlabi/Stegasus.git

In [ ]:
from Stegasus import *

In [ ]:
Typo:Typoceros.Typo = Typoceros.Typo
StringSpans: Typoceros.StringSpans = Typoceros.StringSpans 

In [ ]:
inst = Typo('Hi, How are you?',verbose=True)
(inst.spaces,inst.bits)

## Emojer

In [ ]:
LOONG_TEXT = """Text literals and metacharacters make up this string. The compile function is used to create the pattern."""

In [ ]:
# https://github.com/huggingface/torchMoji

In [ ]:
Emojier : Emojier = Emojier

In [ ]:
tests = 10
print(f"Running {tests} tests")
for i in range(tests):
  data = random_bit_stream(60)
  # text = 'hi, how are you?'
  text = LOONG_TEXT
  verbose = True
  encoded_text,rem = Emojier.encode(text,data,verbose=verbose)
  print('rem=',rem)
  _, deData = Emojier.decode(encoded_text,verbose=verbose)
  deData += rem
  print(f'text="{text}"\n->\nencoded_text="{encoded_text}" \ndata="{data}"\ndeData="{deData}"\ndata==deData="{data==deData}"')
  print(f'ratio={len(data)-len(rem)} / {len(text)}={(len(data)-len(rem)) / len(text)}')
  assert data==deData
  print('\n')
  print("#"*100)
  print('\n')

# 0000

## Mental Deterministic Bot

In [ ]:
#@title Imports
%pip install transformers

### Working solution!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Dict, List, Tuple
import torch
from dataclasses import dataclass


ACTION_SPACE = [ 'ask about kids.', "ask about pets.", 'talk about work.', 
               'ask about marital status.', 'talk about travel.', 'ask about age and gender.',
        'ask about hobbies.', 'ask about favorite food.', 'talk about movies.', 
        'talk about music.', 'talk about politics.']

@dataclass
class Message:
    persona: str
    text: str

class PersonaGPTBot:
    def __init__(self, personas:Dict[str, List[str]], action_space=ACTION_SPACE, model_name="af1tang/personaGPT", use_fast=False):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=use_fast)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        if torch.cuda.is_available():
            self.model = self.model.cuda()
        if 'Bot' not in personas:
          personas['Bot'] = ["My name is RobotMan","I used to be called cliff steel","I was a Nascar Racer"]
        self.personas = dict()
        for k,v in personas.items():
          self.personas[k] = self.tokenizer.encode(''.join(['<|p2|>'] + v + ['<|sep|>'] + ['<|start|>']))
        self.action_space = action_space
        self.dialog_hx = []
        
    def flatten(self, l):
        return [item for sublist in l for item in sublist]

    def to_data(self, x):
        if torch.cuda.is_available():
            x = x.cpu()
        return x.data.numpy()

    def to_var(self, x):
        if not torch.is_tensor(x):
            x = torch.Tensor(x)
        if torch.cuda.is_available():
            x = x.cuda()
        return x
    
    def generate_next(self,
                  bot_input_ids,
                  do_sample=False,  # Default to sampling
                  top_k=10,         # Default to no top-k sampling
                  top_p=.6,        # Default to no top-p sampling
                  temperature=1e-5,   # Default temperature
                  max_length=1000, # Maximum length of generated message
                  pad_token=None   # Token used for padding if pad_token_id is not provided
                  ):
        pad_token = pad_token if pad_token is not None else self.tokenizer.eos_token_id
        
        # Generate a full message using the provided model and parameters
        full_msg = self.model.generate(
            bot_input_ids,
            do_sample=do_sample,
            top_k=top_k,
            top_p=top_p,
            temperature=temperature,
            max_length=max_length,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        # Extract the message from the full generated output and return it
        msg = self.to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
        return msg

    
    def reply(self,user_input,persona:str = "Bot",dialog_hx=None):
        # respond to input 
        dialog_hx= dialog_hx if dialog_hx is not None else self.dialog_hx

        # encode the user input
        user_inp = self.tokenizer.encode(user_input + self.tokenizer.eos_token)
        # append to the chat history
        dialog_hx.append(user_inp)

        # generate a response while limiting the total chat history to 1000 tokens
        bot_input_ids = self.to_var([self.personas[persona] + self.flatten(dialog_hx)]).long()
        msg = self.generate_next(bot_input_ids)
        response = self.tokenizer.decode(msg, skip_special_tokens=True)
        
        return response, dialog_hx

    def ask(self,action:str,dialog_hx=None):
        # respond to input 
        if isinstance(action, int):
          action = self.action_space[action]
        dialog_hx= dialog_hx if dialog_hx is not None else self.dialog_hx
        action_prefix = self.tokenizer.encode(f'<|act|>{action}<|p1|><|sep|><|start|>')
        bot_input_ids = self.to_var([action_prefix + self.flatten(dialog_hx)]).long()
        
        # generate query conditioned on action
        msg = self.generate_next(bot_input_ids)
        
        query = self.tokenizer.decode(msg, skip_special_tokens=True)
        
        return query, dialog_hx
    
    def resume_chat(self, dialog_hx):
        self.dialog_hx = dialog_hx
    
    def converse(self,length, personas:Tuple[str,str], action=None, startMessage:Message=None, dialog_hx=None):
        dialog_hx= dialog_hx if dialog_hx is not None else self.dialog_hx
        if action is None and startMessage is None:
          startMessage = Message(persona=personas[0],text='')
        res = ''
        responses = []
        if action is not None:
          res, dialog_hx = self.ask(action,dialog_hx)
        else:
          res, dialog_hx = self.reply(startMessage.text,startMessage.persona,dialog_hx)
        responses.append(Message(text=res,persona=personas[0]))
        turn = 1
        for i in range(length-1):
          res, dialog_hx = self.reply(res,personas[turn],dialog_hx)
          responses.append(Message(text=res,persona=personas[turn]))
          turn = (turn+1) % 2
        
        return responses , dialog_hx

    


In [ ]:
pg = PersonaGPTBot({'Alice':["I'm a french girl","I love art","my name is Alice"],"Bob" :["I'm a french boy","I love art","my name is Bob"]})

In [ ]:
dia = []
res = [Message(persona='Bob',text='Have you Heard? I\'m getting married!')]

In [ ]:
print(res[-1])

In [ ]:
res, dia = pg.converse(10,('Alice','Bob'),startMessage=res[-1],dialog_hx=dia)
res

In [ ]:
res = None
old_res = None
for i in range(100):
  dia = []
  res = [Message(persona='Bob',text='')]
  res, dia = pg.converse(10,('Alice','Bob'),startMessage=res[-1],dialog_hx=dia)
  if res is not None and old_res is not None and res != old_res:
    print('old_res',old_res)
    print('res',res)
  else:
    print(f'test {i} passed!')
  old_res = res

In [ ]:
a = """[Message(persona='Alice', text='hi how are you doing'),
 Message(persona='Bob', text="i'm good thanks for asking"),
 Message(persona='Alice', text='what do you do for a living'),
 Message(persona='Bob', text='i am a french boy'),
 Message(persona='Alice', text='do you have any hobbies'),
 Message(persona='Bob', text='i like to play sports'),
 Message(persona='Alice', text='what do you like about sports'),
 Message(persona='Bob', text="they're fun to play"),
 Message(persona='Alice', text='do you play often then'),
 Message(persona='Bob', text='yeah i play soccer a lot')]"""

In [ ]:
pg.ask('ask about sports',[])

In [ ]:
pg.reply("hello i'm mary and yes i've a daughter. do you have any children?","Bob", [[31373, 466, 345, 423, 597, 1751, 286, 534, 898, 30, 50256]])

## talk about article

In [ ]:
from transformers import pipeline

# Load the conversational pipeline
chatbot = pipeline("conversational")

# Define the article string
article = "The quick brown fox jumps over the lazy dog."

# Start the conversation
conversation = []
conversation.append("Person 1: Hey, have you read this article?")
conversation.append("Person 2: No, what's it about?")
conversation.append(f"Person 1: It's about a fox jumping over a dog. Here's the article: {article}")

# Generate responses to the article
for i in range(5):
    conversation.append(f"Person 2: {article}")
    response = chatbot(conversation)[-1]["generated_text"]
    conversation.append(f"Person 1: {response}")
    conversation.append(f"Person 2: {response}")
    article_response = chatbot(conversation)[-1]["generated_text"]
    conversation.append(f"Person 1: {article_response}")

# Print the conversation
for line in conversation:
    print(line)


## Putting it all Together

In [ ]:
PersonaGPTBot_Singleton = PersonaGPTBot({'Alice':["I'm a french girl","I love art","my name is Alice"],"Bob" :["I'm a french boy","I love art","my name is Bob"]})

In [ ]:
#@title Hide
# def Hide(bytes:str,message):
#   if set(bytes) > set('10'):
#     raise ValueError(f"bytes:'{bytes}' should only contain 0 and 1")
  
#   message_pipe = [message]
#   bytes_pipe = [bytes]
  
#   stega_bert = masked_stego(message_pipe[-1],bytes_pipe[-1], 3, 0.01)
#   # stego_text': 'The quick red fox jumps over the poor dog.', 'encoded_message':

#   encoded_message = stega_bert['encoded_message']

#   message_pipe.append(stega_bert['stego_text'])
#   bytes_pipe.append(bytes_pipe[-1][len(encoded_message):])

#   stega_typo = Typoceros(message_pipe[-1])
#   enc, rem = stega_typo.encode_encoder(bytes_pipe[-1])

#   bytes_pipe.append(rem)

#   typoed = stega_typo.encode(enc)

#   message_pipe.append(typoed)

#   return {'stego object':message_pipe[-1]
#           , 'remaining bytes':bytes_pipe[-1]
#           , 'encoded bytes':bytes[:-len(bytes_pipe[-1])]
#           ,'bytes_pipe':bytes_pipe
#           ,'message_pipe':message_pipe}
# text = """You can see that it takes the initial inputs x1 and x2 and does the first equation of the reversible networks you learned in Part 3. As you've also learned, the reversible residual has two equations for the forward-pass so doing just one of them will just constitute half of the reversible decoder block. Before doing the second equation (i.e. second half of the reversible residual), it first needs to swap the elements to take into account the stack semantics in Trax. It simply puts x2 on top of the stack so it can be fed to the add block of the half-residual layer. It then swaps the two outputs again so it can be fed to the next layer of the network. All of these arrives at the two equations in Part 3 and it can be used to recompute the activations during the backward pass."""
# # ic(Hide(random_bit_stream(),'Hi, How are you?'))
# # ic(Hide(random_bit_stream(),'Hi, How are you?'))
# # ic(Hide(random_bit_stream(),'Hi, How are you?'))
# # ic(Hide(random_bit_stream(),'Hi, How are you?'))
# # ic(Hide(random_bit_stream(),text))
# # ic(Hide(random_bit_stream(),text))
# # ic(Hide(random_bit_stream(),text))
# # ic(Hide(random_bit_stream(),text))

In [ ]:
#@title Send
# def Send(bytes:str,last_message = '', dialog_hx = []):
#   if set(bytes) > set('10'):
#     raise ValueError(f"bytes:'{bytes}' should only contain 0 and 1")
#   global PersonaGPTBot_Singleton
#   response, dialog_hx = PersonaGPTBot_Singleton.reply(last_message,'Alice',dialog_hx)

  

# Send('111000')

In [ ]:
#@title Pipe
from typing import List, Any, Dict

def pipe(callbacks: List[callable], config: Dict[str, Any]={}, index=0):
    def process_callbacks(state, callbacks: List[callable], config: Dict[str, Any]={}, index=0):
        # Get the current callback
        callback = callbacks[index]

        # Get the next callback (if exists)
        next_callback = None
        if index < len(callbacks) - 1:
            next_callback = lambda s, c, cf=config: process_callbacks(s, callbacks, cf, index + 1)

        # Call the callback with the current state, next callback, and config
        state = callback(state, next_callback, config)

        # Return the final state
        return state

    def _pipe(state):
        return process_callbacks(state, callbacks, config, index)

    return _pipe

def bert_callback(state, next_callback, config):
    if state is None:
        raise ValueError('State is None')
    
    pipe_verbose = config['pipe_verbose']
    encode = config['encode']
    decode = config['decode']
    test = config['test']
    
    if encode or test:
      message_pipe, bytes_pipe = state
      stega_bert = masked_stego(message_pipe[-1],bytes_pipe[-1], 3, 0.01)
      message_pipe.append(stega_bert.encoded_text)
      bytes_pipe.append(stega_bert.remaining_bytes)

    if next_callback is not None:
        state = next_callback(state, next_callback, config)

    if decode or test:
      encoded_text = message_pipe.pop() 
      remaining_bytes = bytes_pipe.pop() 
      encoded_bytes = masked_stego.decode(encoded_text,3,0.01)
      if test:
        assert encoded_bytes + remaining_bytes == bytes_pipe[-1]
      else:
        message_pipe.append(encoded_text)
        bytes_pipe.append(encoded_bytes + remaining_bytes)
    
    return state

def emojer_callback(state, next_callback, config):
    if state is None:
        raise ValueError('State is None')
    
    message_pipe, bytes_pipe = state
    text = message_pipe[-1]
    data = bytes_pipe[-1]
    verbose = config['verbose']
    
    pipe_verbose = config['pipe_verbose']
    encode = config['encode']
    decode = config['decode']

    if encode or test:
      encoded_text,rem = Emojier.encode(text,data,verbose=verbose)
      message_pipe.append(encoded_text)
      bytes_pipe.append(rem)

    if next_callback is not None:
        state = next_callback(state, next_callback, config)
    else:
      print(state)

    if decode or test:
      encoded_pipe_text = message_pipe.pop()
      rem_pipe_bytes = bytes_pipe.pop()

      original_text, deData = Emojier.decode(encoded_pipe_text,verbose=verbose)
      deData += rem_pipe_bytes
      if test:
        assert deData == bytes_pipe[-1]
        assert original_text == message_pipe[-1]
      else:
        message_pipe.append(original_text)
        bytes_pipe.append(deData)
    
    return state

def typo_callback(state, next_callback, config):
    if state is None:
        raise ValueError('State is None')
    
    message_pipe, bytes_pipe = state
    text = message_pipe[-1]
    data = bytes_pipe[-1]
    verbose = config['verbose']
    pipe_verbose = config['pipe_verbose']
    encode = config['encode']
    decode = config['decode']
    test = config['test']

    if pipe_verbose:
      print(state)
    if encode or test:
      typo = Typo(text,verbose=verbose)

      chs_ints, rem = typo.encode_encoder(data)
      encoded_text = typo.encode(chs_ints) 
      
      message_pipe.append(encoded_text)
      bytes_pipe.append(rem)

    if next_callback is not None:
        state = next_callback(state, next_callback, config)

    if decode or test:
      encoded_pipe_text = message_pipe.pop()
      rem_pipe_bytes = bytes_pipe.pop()

      origina_string, values = typo.decode(encoded_pipe_text)
      if test:
        assert origina_string == text
        assert values = data
      else:
        message_pipe.append(origina_string)
        bytes_pipe.append()
    
    
    return state

callbacks = [bert_callback, emojer_callback,typo_callback]
# callbacks = [typo_callback]

# Apply the function with an initial state
initial_state = [['Hi, How are you?'],[random_bit_stream(30)]]
p = pipe(callbacks, {"verbose": False,"pipe_verbose": False,"encode":True,"decode":False,"test":False})
mq, bq = p(initial_state)

print(mq[-1],bq[-1]) 



In [ ]:
def StegasusEncode(text,bytes_str):
  initial_state = [[text],[bytes_str]]
  callbacks = [bert_callback, emojer_callback,typo_callback]
  p = pipe(callbacks, {"verbose": False,"pipe_verbose": False,"encode":True,"decode":False,"test":False})
  mq, bq = p(initial_state)
  return (mq[-1],bq[-1]) 

def StegasusDecode(text):
  initial_state = [[text],['']]
  callbacks = [bert_callback, emojer_callback,typo_callback]
  p = pipe(callbacks, {"verbose": False,"pipe_verbose": False,"encode":False,"decode":True,"test":False})
  mq, bq = p(initial_state)
  return (mq[-1],bq[-1]) 
  

def StegasusTest(text):
  initial_state = [[text],[random_bit_stream(len(text))]]
  callbacks = [bert_callback, emojer_callback,typo_callback]
  p = pipe(callbacks, {"verbose": False,"pipe_verbose": False,"encode":False,"decode":True,"test":False})
  mq, bq = p(initial_state)
  return (mq[-1],bq[-1]) 

In [ ]:
LOONG_TEXT = '''Metaphysical solipsism is a variety of solipsism. Based on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality and that all other realities, including the external world and other persons.'''

In [ ]:
Famous_Demo = 'Hi, How are you?'

In [ ]:
t = Typo(LOONG_TEXT)
t.spaces

In [ ]:
Typo.

In [ ]:
xtx = Famous_Demo
data = random_bit_stream(len(xtx))
returned = StegasusEncode(xtx,data)

In [ ]:
returned